# Hardware manager

In [1]:
import numpy as np
from PIL import Image
import socket

In [2]:
max_data_size = 400000

In [3]:


img = Image.open('resized_image_256.jpg')
resized_image = img.resize((256, 256), Image.ANTIALIAS)

from PIL import ImageOps
resized_image = ImageOps.grayscale(resized_image)
resized_image.show()
data = np.asarray(resized_image)
# print(data)
# print(data.shape)
# with open('./image.txt', 'r') as f:
#     lines = f.readlines()

# # Convert lines into a 2D list
# data = [[int(num) for num in line.split()] for line in lines]

In [4]:
data = np.reshape(np.array(data), (1, 256 * 256))
np.set_printoptions(threshold=np.inf)
data = np.array2string(data)

In [5]:
import socket
from struct import unpack

def start_server(host, port):
    # host = '10.1.212.147'
    # port = 64505  # Make sure this port is not in use

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)
    print('Server started. Waiting for connections...')
    server_running = True
    
    while server_running:
        client_socket, addr = server_socket.accept()
        print(f'Connection from: {str(addr)}')
        while True:
            bs = client_socket.recv(8)
            (length,) = unpack('>Q', bs)
            print(length)
            recv_data = b''
            while len(recv_data) < length:
                to_read = length - len(recv_data)
                recv_data += client_socket.recv(
                    4096 if to_read > 4096 else to_read)
            # recv_data = receive_all(client_socket, image_data_size)
            if not recv_data:
                break
            print(f'From connected user: {recv_data[:1024]} ...')
            data = '202'
            client_socket.send(data.encode())
            
            server_running = False
            break

        client_socket.close()
    server_socket.close()
    print("close connection")
    return recv_data

In [6]:
from struct import pack

def start_client(host, port, message):
    # host = '10.1.212.148'
    # port = 64501  # Must be the same as the server port

    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    message = message.encode()
    length = pack('>Q', len(message)) 
    print(length)
    client_socket.sendall(length) #send length
    client_socket.sendall(message) #send data
    #receive resp from server
    data = client_socket.recv(3)
    print(f'Received from server: {data}')
    
    client_socket.close()
    return data

### Send data to FPGA 1

In [7]:


gamma = [0, 50]
gamma = np.reshape(np.array(gamma), (2))
gamma = np.array2string(gamma)
recv = start_client('10.1.212.148', 64502, gamma)

b'\x00\x00\x00\x00\x00\x00\x00\x07'
Received from server: b'202'


### Send data to FPGA 2

In [8]:
recv = start_client('10.1.212.147', 64506, data)

b'\x00\x00\x00\x00\x00\x04\x1cs'
Received from server: b'202'


In [9]:
print(len(data))

269427


### Waiting for the results

In [10]:
recv = start_server('10.1.212.148', 64510)

Server started. Waiting for connections...
Connection from: ('10.1.212.147', 46608)
265785
From connected user: b'[110 110 110 110 110 110 110 110 111 111 111 112 112 113 113 113 114 113\n 113 113 113 114 115 116 117 117 117 117 117 117 117 117 118 118 118 117\n 116 115 113 112 111 110 110 110 110 111 112 113 113 114 115 116 117 117\n 117 117 118 118 118 118 118 118 118 118 119 119 119 118 117 116 115 114\n 112 111 111 111 111 112 113 114 114 114 114 114 113 112 111 110 111 111\n 111 111 111 111 111 111 111 111 111 112 112 113 113 113 112 112 114 115\n 117 118 119 120 123 123 125 126 125 124 123 122 122 121 119 117 115 114\n 114 114 114 114 113 113 113 112 112 112 112 111 111 111 111 112 113 114\n 115 115 115 116 116 117 117 117 117 117 117 117 117 117 117 117 114 114\n 114 114 114 114 114 114 115 115 115 116 116 117 117 117 117 117 117 117\n 117 117 117 117 116 116 116 115 115 114 114 114 115 115 115 115 115 115\n 115 115 114 114 114 114 114 114 114 114 114 114 114 115 117 119 121 122

In [11]:
s = recv.decode()
lines = s.strip('[]').split('\n')
data = [int(num) for line in lines for num in line.split()]

In [12]:
print(len(data))

65536
